In [19]:
import json
from collections import Counter, defaultdict
from wiki_labels import qid_lab_get
from wiki_location import q2cc
from text_utils import cl
import sys
from tqdm.notebook import tqdm
import gzip
sys.path.insert(0, '/projekti/mondoAPI')
from pnu.text_utils import norm_pname
from pnu.detect_lang_scr import get_provenance

WIKI_DIR = '/backup/wikidata'


In [2]:
# wiki lang to iso
# TODO make separate script
WIKI_DIR = '/backup/wikidata'
api_key = open('airtable_key.txt').read()
from pyairtable import Table, Api
import sys
sys.path.insert(0, '/projekti/nelma')
from mondoDB.referencedb import provi

if True:
    lang2cc = defaultdict(Counter)
    for k, v in provi.items():
        lang, s, c = v['id'].split('_')
        bod = float(v['Percent'])*3 if 'Percent' in v else 0
        if 'Status' in v:
            if v['Status'] == 'official':
                bod += 1
            elif v['Status'] == 'official_regional':
                bod += 0.5
            elif v['Status'] == 'de_facto_official':
                bod += 0.9
            elif v['Status'] == 'romanized':
                bod += 0.2
            elif v['Status'] == 'foreign':
                bod += -0.5
        if bod>1.1:
            lang2cc[f'{lang}'][c] = max(bod, lang2cc[f'{lang}'][c])
    api = Api(api_key)
    w2iso = {t['fields']['WMF']: t['fields']['qid'] if 'qid' in t['fields'] else None for t in Table(api_key, 'appUZvAm9EHZgC1Eg', 'wiki_lang').all()}
    wiki2cc = defaultdict(Counter)
    for k, v in w2iso.items():
        if len(k) == 2:
            wiki2cc[v].update(lang2cc[k])
        elif k.count('-') == 1:
            a, b = k.split('-')
            if len(a) == 2 and len(b) == 2:
                wiki2cc[v][b.upper()] += 5 
            elif len(a) == 2 and len(b) == 4:
                #wiki2cc[v].update(lang2cc[f'{k}_{b.uppser()}'])
                wiki2cc[v].update(lang2cc[a])
    wikil2cc = {k: {cc: round(100*v2/max(v.values())) for cc, v2 in v.items()} for k, v in wiki2cc.items() if len(v)>0 and max(v.values())>0}

cc2lang = defaultdict(set)
for lang, ccs in lang2cc.items():
    for cc in ccs:
        if ccs[cc]>1:
            cc2lang[cc].add(lang)
iso2w = {k:v for v, k in w2iso.items()}

wikil2cc['Q1860']['US'], cc2lang['CH']

(94, {'de', 'en', 'fr', 'it'})

In [22]:
fo = gzip.open('/projekti/data/companies/mondo_comp_wiki.csv.gz', 'wt')

for l in open(f'{WIKI_DIR}/wiki_org.jsonl'):
    j = json.loads(l)
    qid = int(j['id'][1:])
    ccs = Counter()
    for q in j['location']:
        if q in q2cc:
            ccs[q2cc[q]] += 1
    if len(ccs) > 0:
        cc = ccs.most_common()[0][0]
        if cc not in cc2lang:
            raise NotImplementedError
        labs = {}
        # use companies names for languages in company country
        for lang in cc2lang[cc]:
            labs.update(qid_lab_get(qid, lang=lang, include_alt=True))
        for l in labs:
            prov,_ = get_provenance(l, cc)
            if prov:
                tow = f'{l}\torg\t{prov}\n'
                fo.write(tow)
fo.close()            

In [14]:
labs

{'sabmiller': {'af',
  'bg',
  'ca',
  'cs',
  'da',
  'de',
  'el',
  'en',
  'en-ca',
  'en-gb',
  'es',
  'fi',
  'fr',
  'hu',
  'id',
  'it',
  'lmo',
  'mk',
  'ms',
  'nb',
  'nl',
  'oc',
  'pl',
  'pt',
  'pt-br',
  'ro',
  'ru',
  'simple',
  'sk',
  'sv',
  'tr',
  'uk',
  'vec'},
 'sab miller': {'en', 'es', 'it', 'ro'},
 'sabmiller group': {'en'},
 'sabmiller plc': {'en', 'hu'}}